# Work‑stream 4 — Impairment Scoring Agent (Notebook Version)

This notebook mirrors the `score.py` Lambda we might build. It shows, step‑by‑step, 
how the Strands agent ingests the JSON payload from the detection agent (Work‑stream 3), 
looks up scoring guidelines from the Knowledge Base, calculates a score for each 
impairment, and then returns a final aggregated score.

> **Why a notebook?**  
> • Allows for easy modification of the input payload to test different scenarios.  
> • Lets you tweak the prompt or tools and run a sample without redeploying a Lambda.  
> • Serves as executable documentation for future hand‑offs.


In [26]:
import os, json, boto3, re
import numpy as np
from collections import defaultdict
from strands import Agent, tool

# ---- Set these before running locally ----
# Knowledge base configuration - uncomment the line below to use Bedrock Knowledge Base instead of local files
kb_id = 'YSWIGPQHRJ'
# kb_id = None  # Reset to None to ensure clean state

model_id = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
embedding_model_id = 'amazon.titan-embed-text-v2:0'

# Local knowledge base path
local_kb_path = "../underwriting_manual"

# Initialize the payload for easy modification
impairments_payload = [
  {
    "impairment_id": "hypertension",
    "scoring_factors": {
      "blood_pressure": "128/92 mmHg",
      "age": 41,
      "medication": "Lisinopril 10mg",
      "duration": "At least since 2022-04-18",
      "compliance": "Good - regular refills",
      "target_organ_damage": "None evident",
      "comorbidities": "None evident",
      "family_history": "Father had heart attack at age 58"
    },
    "evidence": [
      "Rx: Lisinopril 10mg for hypertension, filled 2024-01-10 (90 tablets)",
      "Rx: Lisinopril 10mg for hypertension, filled 2023-10-12 (90 tablets)",
      "MIB: Code 311C 'CARDIOVASCULAR - HYPERTENSION TREATED' from 2022-04-18",
      "Application: Self-reported Lisinopril 10mg for blood pressure",
      "Application: Blood pressure reading 128/92 mmHg"
    ]
  }
]


In [27]:
# Local Knowledge Base Setup
local_kb_store = None
bedrock_runtime = boto3.client('bedrock-runtime')

def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def create_embedding(text):
    """Create embedding using Amazon Titan model"""
    response = bedrock_runtime.invoke_model(
        modelId=embedding_model_id,
        body=json.dumps({"inputText": text})
    )
    embedding = json.loads(response['body'].read())['embedding']
    return np.array(embedding)

def load_local_knowledge_base():
    """Load markdown files from local underwriting manual and create embeddings"""
    global local_kb_store
    
    if 'kb_id' in globals() and kb_id is not None:
        print("Bedrock KB configured, skipping local KB loading...")
        return
    
    print(f"Loading local knowledge base from {local_kb_path}...")
    
    kb_documents = []
    
    # Find all markdown files in the underwriting manual directory
    if not os.path.exists(local_kb_path):
        print(f"Warning: Local KB path {local_kb_path} does not exist")
        return
    
    for filename in os.listdir(local_kb_path):
        if filename.lower().endswith('.md'):
            file_path = os.path.join(local_kb_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                print(f"✓ Loading {filename} ({len(content)} chars)")
                
                # Create embedding for the document
                embedding = create_embedding(content)
                
                kb_documents.append({
                    'filename': filename,
                    'content': content,
                    'embedding': embedding
                })
                
            except Exception as e:
                print(f"✗ Error loading {filename}: {e}")
    
    local_kb_store = kb_documents
    print(f"Local knowledge base loaded with {len(kb_documents)} documents")

# Load the local knowledge base if kb_id is not defined or None
if 'kb_id' not in globals() or kb_id is None:
    load_local_knowledge_base()
else:
    print("Using Bedrock Knowledge Base")


Using Bedrock Knowledge Base


In [28]:
kb_rt = boto3.client('bedrock-agent-runtime')

@tool
def kb_search(canonical_term: str):
    """Return markdown for the top KB hit from either local or Bedrock knowledge base."""
    
    if ('kb_id' not in globals() or kb_id is None) and local_kb_store:
        # Use local knowledge base
        print(f"Searching local KB for: {canonical_term}")
        
        # Create embedding for the search query
        query_embedding = create_embedding(canonical_term)
        
        # Find the most similar document
        best_match = None
        best_similarity = -1
        
        for doc in local_kb_store:
            similarity = cosine_similarity(query_embedding, doc['embedding'])
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = doc
        
        if best_match:
            print(f"Best match: {best_match['filename']} (similarity: {best_similarity:.3f})")
            return best_match['content']
        else:
            return "No matching documents found in local knowledge base."
    
    else:
        # Use Bedrock Knowledge Base
        print(f"Searching Bedrock KB for: {canonical_term}")
        resp = kb_rt.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': canonical_term},
            retrievalConfiguration={'vectorSearchConfiguration': {'numberOfResults': 1}}
        )
        print(resp)
        # According to official AWS documentation, the field is 'text', not 'text_markdown'
        return resp['retrievalResults'][0]['content']['text']

@tool
def calculator(values: list[float]):
    """Calculates the sum of a list of numbers. Use this for adding up credits (negative numbers) and debits (positive numbers)."""
    print(f"Calculator adding: {values}")
    return sum(values)


In [29]:
PROMPT = """You are a senior life insurance underwriter specializing in risk assessment scoring. Your job is to calculate a risk score for an application based on a list of identified impairments and their scoring factors.

You will be given a JSON array of impairments. For each impairment in the input list, you must perform the following steps in sequence:

1. **Lookup**: Call the `kb_search` tool using the impairment's `impairment_id` as the `canonical_term`. This returns the authoritative underwriting manual section.

2. **Analyze**: Carefully read the returned markdown. Use the `scoring_factors` provided for the impairment to find the correct debits and credits in the rating tables. For example, a `blood_pressure` of "128/92 mmHg" and `age` of 41 falls into the "141-150/91-95" row for the "Age 40-60" column in the hypertension manual, which indicates a debit between +25 and +50. Use the lower value if a range is given.

3. **Calculate Subtotal**: Create a list of all numerical debits (positive numbers) and credits (negative numbers) you identified. Pass this list to the `calculator` tool to get a `sub_total` for the impairment.

4. **Explain**: After calculating the subtotal, you must generate a detailed `reason` string explaining exactly how you arrived at that score, citing the specific scoring factors, table values, and modifying factors used.

Repeat this entire process for every impairment in the input list.

Once you have a `sub_total` for all impairments, create a final list containing all the individual sub-totals. Call the `calculator` tool one last time with this list to get the final `total_score`.

Finally, structure your entire response as a single JSON object. Do not include any other text or explanation outside of the final JSON block.

Your output must be in this exact format:
```json
{
  "total_score": 100,
  "impairment_scores": [
    {
      "impairment_id": "hypertension",
      "sub_total": 50,
      "reason": "Debit of +25 for BP 128/92 at age 41. Debit of +25 for newly diagnosed. No credits applied."
    }
  ]
}
```
"""


In [30]:
# Create the scoring agent with the correct tools
scoring_agent = Agent(
    system_prompt=PROMPT,
    tools=[kb_search, calculator],
    model=model_id,
)


In [31]:
def run_scoring(payload):
    """Utility to run the scoring agent in‑notebook"""
    
    # Create a string message with the JSON payload
    message = f"Here is the JSON payload of impairments to score:\\n\\n{json.dumps(payload, indent=2)}"
    
    print("Sending payload to the scoring agent...")
    
    # Call the agent with the message
    res = scoring_agent(message)
    print("Agent response:")
    print(res)
    
    # Extract JSON from between ```json ... ``` tags if present
    res_str = res.__str__()
    json_match = re.search(r"```json\\s*(.*?)\\s*```", res_str, re.DOTALL)
    if json_match:
        res_str = json_match.group(1)
    return json.loads(res_str)


In [ ]:
# Run the scoring agent using the input payload
print("=== Running Impairment Scoring Agent ===\\n")

try:
    results = run_scoring(impairments_payload)
    
    print("\\n=== Final Scoring Results ===")
    print(json.dumps(results, indent=2))
    
except Exception as e:
    print(f"\\n\\nError running scoring: {e}")
    print("\\nMake sure:")
    print("1. Your AWS credentials are configured")
    print("2. The kb_id and model_id are set correctly")
    print("3. You have access to the specified Bedrock model")


=== Running Impairment Scoring Agent ===\n
Sending payload to the scoring agent...
I'll help you calculate a risk score for this hypertension impairment by following the required steps.

First, let me look up the hypertension impairment in the knowledge base.
Tool #1: kb_search
Searching Bedrock KB for: hypertension
{'ResponseMetadata': {'RequestId': 'fb244024-d6b7-4a8f-a711-06336d5b7a29', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 30 Jul 2025 17:32:34 GMT', 'content-type': 'application/json', 'content-length': '8096', 'connection': 'keep-alive', 'x-amzn-requestid': 'fb244024-d6b7-4a8f-a711-06336d5b7a29'}, 'RetryAttempts': 0}, 'retrievalResults': [{'content': {'text': "# Hypertension\n\n## Definition & Classification\n\n**Hypertension**: A chronic medical condition characterized by persistently elevated blood pressure in the arteries. Hypertension significantly increases the risk of heart disease, stroke, kidney disease, and other health problems.\n\n### Classification System